# keyword extraction 

## install package & import

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%%capture
!pip install ko-sentence-transformers
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

In [3]:
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Mecab
from pathlib import Path
from transformers import FeatureExtractionPipeline
from transformers import BertTokenizer
from transformers import BertConfig, BertModel
from sklearn.feature_extraction.text import CountVectorizer
from kobert_tokenizer import KoBERTTokenizer
import numpy as np
import itertools
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, util
import numpy as np

In [4]:
%cd 

/root


## load dataset

In [5]:
%cd /content/drive/Othercomputers/skt 노트북/chat_summary
# %cd /content/drive/MyDrive/chat_summary
from load_dataset import *

/content/drive/Othercomputers/skt 노트북/chat_summary


In [6]:
news = News(data_dir='newsData')

docs, topics = news.load_data()

## tokenizer 
- mecab 

In [ ]:
class CustomTokenizer:
    def __init__(self, tagger, tag):
        self.tagger = tagger
        self.tag = tag

    def __call__(self, sent):
        if self.tag == 'nouns':
            word_tokens = self.tagger.nouns(sent)
        elif self.tag == 'morphs':
            word_tokens = self.tagger.morphs(sent)
        elif self.tag == 'NNP':
            word_tokens = list(map(lambda x: x[0], filter(lambda x: x[1] == 'NNP', self.tagger.pos(sent))))
        else:
            raise Exception('keyerror')
        result = [word for word in word_tokens if len(word) > 1]
        return result

In [ ]:
custom_tokenizer = CustomTokenizer(Mecab(), tag='NNP')

In [ ]:
n_gram_range = (3, 3)
stop_words = "english"

# count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit([doc])
count = CountVectorizer(tokenizer=custom_tokenizer, ngram_range=(1, 1)).fit(docs)
candidates = count.get_feature_names_out()

print('trigram 개수 :',len(candidates))
print('trigram 다섯개만 출력 :',candidates[:5])

/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


trigram 개수 : 6085
trigram 다섯개만 출력 : ['가교' '가너' '가닉' '가델라' '가디']


In [ ]:
candidates.shape

(6085,)

## sentence embedding
- ko-sentence-embedding

In [ ]:
model = SentenceTransformer("jhgan/ko-sbert-nli")
doc_embedding = model.encode(docs)
candidate_embeddings = model.encode(candidates)

In [ ]:
def max_sum_sim(doc_embedding, candidate_embeddings, words, top_n, nr_candidates):
    # 문서와 각 키워드들 간의 유사도
    distances = cosine_similarity(doc_embedding, candidate_embeddings)

    # 각 키워드들 간의 유사도
    distances_candidates = cosine_similarity(candidate_embeddings, 
                                            candidate_embeddings)

    # 코사인 유사도에 기반하여 키워드들 중 상위 top_n개의 단어를 pick.
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [candidates[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    # 각 키워드들 중에서 가장 덜 유사한 키워드들간의 조합을 계산
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]

In [ ]:
max_sum_sim(doc_embedding[2:3], candidate_embeddings, candidates, top_n=5, nr_candidates=10)

['김성태', '정세은', '북미', '문재인', '한미정상회담']

In [ ]:
docs[2]

'홍준표 "文정부, 얼마나 사정했으면 지선 하루전 북미회담…" 지선 하루 앞 북미정상회담…한국당, 복잡한 \'속마음\' \'지선 악재\' 전망…경제이슈 전력투구도 쉽지 않아 (서울=뉴스1) 이후민 기자 = 북미정상회담이 6·13 지방선거 하루 전인 다음달 12일 싱가포르에서 개최되는 가운데 이를 지켜보는 자유한국당의 입장이 복잡하게 됐다. 북미정상회담 결과에 따라 지난 남북정상회담에서 연출됐던 것과 같은 평화무드가 한반도를 휩쓸면 절대적으로 정부여당에 유리할 수 밖에 없기 때문이다. 한국당은 그동안 \'완전한 핵폐기\'를 주장하며 4·27 남북정상회담을 \'위장평화 쇼\' \'남북회담 쇼\'로 규정해왔다. 하지만 북미가 정상회담을 통해 \'완전한 비핵화\'에 대한 방법론에 전격 합의할 경우 그 내용과는 별도로 평화무드가 지방선거까지 이어질 수 있다는 경계심이 적지 않은 상황이다. 홍준표 한국당 대표도 지난 11일 열린 한국당 경북도당 필승결의대회에서 "북미회담이 북핵폐기로 가는 마지막 기회가 될 수 있다"며 "미국 측에 한국의 제1야당 입장이 담긴 공개서한을 전달하려고 한다"고 한 바 있다. 공개 서한에 대해서는 김무성 의원이 이끄는 당 내 북핵폐기추진특별위원회에서 서한의 내용과 전달 방식 등을 논의할 예정이다. 홍 대표도 대구에서 열린 필승결의대회에서 "문재인 정부가 얼마나 사정을 했으면 지방선거 하루 전에 싱가포르에서 회담을 하느냐"며 "결국은 남북평화쇼로 지방선거를 덮어버리겠다는 것"이라고 주장하기도 했다. 특히 도널드 트럼프 미국 대통령 역시 이번 11월 중간선거 결과에 반영될 수 있는 이번 회담 성과에 심혈을 기울일 수밖에 없는 만큼, 미국과 북한 모두 그간의 모습과는 달리 전향적이고 파격적인 태도를 보일 수 있지 않겠느냐는 관측도 나오고 있다. 다만 한국당은 앞선 2000년, 2007년의 남북정상회담이 총선과 대선 등 큰 선거에 영향을 주지 않았던 선례가 있는 만큼 지나치게 비관할 필요는 없다는 낙관론을 지도부가 나서 주입하다시피 하는 형편이다. 홍

In [ ]:
doc_embedding.shape

(1600, 768)

In [ ]:
candidate_embeddings.shape

(6085, 768)

In [ ]:
def mmr(doc_embedding, candidate_embeddings, words, top_n, diversity):

    # 문서와 각 키워드들 간의 유사도가 적혀있는 리스트
    word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)

    # 각 키워드들 간의 유사도
    word_similarity = cosine_similarity(candidate_embeddings)

    # 문서와 가장 높은 유사도를 가진 키워드의 인덱스를 추출.
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # keywords_idx = [2]
    keywords_idx = [np.argmax(word_doc_similarity)]

    # 가장 높은 유사도를 가진 키워드의 인덱스를 제외한 문서의 인덱스들
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # ==> candidates_idx = [0, 1, 3, 4, 5, 6, 7, 8, 9, 10 ... 중략 ...]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    # 최고의 키워드는 이미 추출했으므로 top_n-1번만큼 아래를 반복.
    # ex) top_n = 5라면, 아래의 loop는 4번 반복됨.
    for _ in range(top_n - 1):
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # MMR을 계산
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # keywords & candidates를 업데이트
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return [words[idx] for idx in keywords_idx]

In [ ]:
mmr(doc_embedding[1:2], candidate_embeddings, candidates, top_n=5, diversity=0.2)

['김경수', '시사평론가', '뉴스룸', '조선일보', '드루']

In [ ]:
max_sum_sim(doc_embedding[1:2], candidate_embeddings, candidates, top_n=5, nr_candidates=10)

['김동준', '김무성', '뉴스룸', '드루', '시사평론가']

In [ ]:
docs[1]

'[종합]김경수, 드루킹 \'옥중편지\'에 "황당하고 어처구니 없는 소설" "거리낄 게 있다면 특검 주장하고 선거 나왔겠나" "선거판 흔들 수 있다 생각했다면 날 잘못 본 것" 【서울=뉴시스】이근홍 기자 = 김경수 더불어민주당 경남지사 후보는 18일 민주당원 댓글 조작 사건의 핵심 인물인 \'드루킹\' 김동원 씨가 \'김 후보에게 매크로 프로그램을 직접 보여줬다\'는 내용을 담은 옥중편지를 조선일보에 보낸 것과 관련 "황당하고 어처구니 없는 소설같은 얘기"라고 밝혔다. 김 후보는 이날 부산 영주동 부산민주공원에서 기자들과 만나 "이렇게 소설 같은 얘기를 바로 기사화해도 되는 건지 묻지 않을 수 없다"며 이같이 말했다. 그는 "드루킹이 검찰에 검은 거래까지 제안을 했다는데 의도가 뻔하다"며 "이런 얘기를 바로 기사화하고 있는 조선일보는 (드루킹과) 한 팀인지 묻지 않을 수 없다"고 지적했다. 김 후보는 "제가 거리낄 게 있다면 경찰 조사를 먼저 받겠다고 하고, 특검도 먼저 주장하고 그리고 이번 지방선거에 나섰겠나"라며 "(드루킹 사건으로) 선거판을 흔들 수 있다고 생각한다면 그건 저를 그리고 경남도민을 잘못 본 것"이라고 강조했다. 그는 "이번 선거는 우리 정치를 낡은 정치, 구태 정치로 다시 되돌리려는 과거팀과 새로운 미래를 열어가려는 미래팀의 대결임이 분명해졌다"며 "저는 경남도민과 함께 절대 물러서지 않고 반드시 이번 지방선거를 승리로 이끌겠다"고 덧붙였다. 김 후보 캠프의 대변인을 맡고 있는 제윤경 민주당 의원도 이날 논평을 통해 "드루킹은 협박과 댓글공작으로 정치인에게 접근한 정치브로커에 불과하다"며 "정치브로커의 황당소설에 속을 국민은 없다"고 주장했다. 그는 "조선일보가 보도한 드루킹 옥중편지는 검찰이 자신에 대한 수사 축소와 빠른 석방을 보장하면 김 후보의 댓글 지시에 대해 진술하겠다는 제안이 받아들여지지 않자 작성된 것에 불과하다"며 "조선일보는 계속적으로 김 후보와 드루킹을 연관시키면서 이번 지방선거에 개입하려는 행동을 즉각 중단해야 한다"고 

## modularization

In [7]:
class CustomTokenizer:
    def __init__(self, tagger, tag):
        self.tagger = tagger
        self.tag = tag

    def __call__(self, sent):
        if self.tag == 'nouns':
            word_tokens = self.tagger.nouns(sent)
        elif self.tag == 'morphs':
            word_tokens = self.tagger.morphs(sent)
        elif self.tag == 'NNP':
            word_tokens = list(map(lambda x: x[0], filter(lambda x: x[1] == 'NNP', self.tagger.pos(sent))))
        else:
            raise Exception('keyerror')
        result = [word for word in word_tokens if len(word) > 1]
        return result

class KeyBERT:
    def __init__(self, tag='NNP'):
        self.model = SentenceTransformer("jhgan/ko-sbert-nli")
        self.tokenizer = CustomTokenizer(Mecab(), tag=tag)

    def mss_(self, doc_embedding, candidate_embeddings, words, top_n, nr_candidates):
        # 문서와 각 키워드들 간의 유사도
        distances = cosine_similarity(doc_embedding, candidate_embeddings)

        # 각 키워드들 간의 유사도
        distances_candidates = cosine_similarity(candidate_embeddings, 
                                                candidate_embeddings)

        # 코사인 유사도에 기반하여 키워드들 중 상위 top_n개의 단어를 pick.
        words_idx = list(distances.argsort()[0][-nr_candidates:])
        words_vals = [candidates[index] for index in words_idx]
        distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

        # 각 키워드들 중에서 가장 덜 유사한 키워드들간의 조합을 계산
        min_sim = np.inf
        candidate = None
        for combination in itertools.combinations(range(len(words_idx)), top_n):
            sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
            if sim < min_sim:
                candidate = combination
                min_sim = sim

        return [words_vals[idx] for idx in candidate]
    
    
    def mmr_(self, doc_embedding, candidate_embeddings, words, top_n, diversity):

        # 문서와 각 키워드들 간의 유사도가 적혀있는 리스트
        word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)

        # 각 키워드들 간의 유사도
        word_similarity = cosine_similarity(candidate_embeddings)

        # 문서와 가장 높은 유사도를 가진 키워드의 인덱스를 추출.
        # 만약, 2번 문서가 가장 유사도가 높았다면
        # keywords_idx = [2]
        keywords_idx = [np.argmax(word_doc_similarity)]

        # 가장 높은 유사도를 가진 키워드의 인덱스를 제외한 문서의 인덱스들
        # 만약, 2번 문서가 가장 유사도가 높았다면
        # ==> candidates_idx = [0, 1, 3, 4, 5, 6, 7, 8, 9, 10 ... 중략 ...]
        candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

        # 최고의 키워드는 이미 추출했으므로 top_n-1번만큼 아래를 반복.
        # ex) top_n = 5라면, 아래의 loop는 4번 반복됨.
        for _ in range(top_n - 1):
            candidate_similarities = word_doc_similarity[candidates_idx, :]
            target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

            # MMR을 계산
            mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
            mmr_idx = candidates_idx[np.argmax(mmr)]

            # keywords & candidates를 업데이트
            keywords_idx.append(mmr_idx)
            candidates_idx.remove(mmr_idx)

        return [words[idx] for idx in keywords_idx]

    def predict(self, doc, ngram_range=(1,1), mode='mmr', top_n=5, nr_candidates=10, diversity=0.2):  # **kwargs: 'nr_candidates': 10, 'diversity': 0.2
        count = CountVectorizer(tokenizer=self.tokenizer, ngram_range=ngram_range).fit([doc])
        candidates = count.get_feature_names_out()

        doc_embedding = self.model.encode([doc])
        candidate_embeddings = self.model.encode(candidates)

        keyword = None
        if mode == 'mss':
            keyword = self.mss_(doc_embedding, candidate_embeddings, candidates, top_n=top_n, nr_candidates=nr_candidates)
        elif mode == 'mmr':
            keyword = self.mmr_(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=diversity)
        else:
            raise Exception('keyerror')

        return keyword

In [9]:
keybert = KeyBERT()

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.46k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/620 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/123 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/495k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/538 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/248k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [10]:
keybert.predict(docs[0])

/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


['민주당', '북미', '문재인', '한반도', '국당']

In [14]:
tokenizer = CustomTokenizer(tagger=Mecab(), tag='NNP')

In [16]:
mecab = Mecab()
mecab.pos(docs[0])

[('하필', 'MAG'),
 ('지방', 'NNG'),
 ('선거', 'NNG'),
 ('전날', 'NNG'),
 ('북미', 'NNP'),
 ('회담', 'NNG'),
 ('…', 'SE'),
 ('야당', 'NNG'),
 ('엔', 'JKB+JX'),
 ('초대', 'NNG'),
 ('형', 'XSN'),
 ('악재', 'NNG'),
 ('?', 'SF'),
 ('한국', 'NNP'),
 ('당', 'NNG'),
 ('“', 'SSO'),
 ('의도', 'NNG'),
 ('개입', 'NNG'),
 ('된', 'XSV+ETM'),
 ('날짜', 'NNG'),
 ('선정', 'NNG'),
 ('”', 'SSC'),
 ('민주당', 'NNP'),
 ('“', 'SSO'),
 ('평화', 'NNG'),
 ('여', 'NP'),
 ('는', 'JX'),
 ('역사', 'NNG'),
 ('적', 'XSN'),
 ('계기', 'NNG'),
 ('”', 'SSC'),
 ('6', 'SN'),
 ('·', 'SC'),
 ('13', 'SN'),
 ('지방', 'NNG'),
 ('선거', 'NNG'),
 ('전날', 'NNG'),
 ('인', 'VCP+ETM'),
 ('다음', 'NNG'),
 ('달', 'NNG'),
 ('12', 'SN'),
 ('일', 'NNBC'),
 ('에', 'JKB'),
 ('북', 'NNP'),
 ('·', 'SC'),
 ('미', 'NNG'),
 ('정상', 'NNG'),
 ('회담', 'NNG'),
 ('개최', 'NNG'),
 ('가', 'JKS'),
 ('확정', 'NNG'),
 ('되', 'XSV'),
 ('자', 'EC'),
 ('여야', 'NNG'),
 ('의', 'JKG'),
 ('표정', 'NNG'),
 ('은', 'JX'),
 ('크', 'VA'),
 ('게', 'EC'),
 ('엇갈렸', 'VV+EP'),
 ('다', 'EF'),
 ('.', 'SF'),
 ('더불', 'VV'),
 ('어', 'EC'),
 ('민주당', 